<a href="https://colab.research.google.com/github/TongSii/hds5210-2025/blob/main/week11/week11_assignment_coding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 11 Assignment


Please do the programming exercise and verify that your code works using the tests, then think about your final project and fill out the questions in the second part.

---
---

### 47.1: Filtering and summarizing data

For this work, you'll find a data file complications_all.csv om Canvas week11.

Read in the data file and create a variable called `mo_hospitals` that contains a data frame from the `complications_all.csv` file, filtered down to only contain those hospitals from the state of Missouri (MO).

Then aggregate that data by hospital into a variable named `mo_summary`.  There are some key fields that we want to summarize:
* We want to know the earliest date that each hospital was participating in any program
* We want to know the latest date that each hospital stopped participating in any program
* We want to know the total number of patients in the denominators of these programs

Some things to note:
* You will need to convert the `Start Date` and `End Date` to actual datetime fields
* You will need to clean up and convert the `Denominator` field to just be numeric - the rule that you should use it to simply remove any records where the `Denominator` is `'Not Available'`


The final result of this step should be a new data frame called `mo_summary` that contains one row for each hospital and contains the min start date, max end date, and total denominator.  Use the names `start_date`, `end_date`, and `number` for those columns in `mo_summary`.


You do not need to create your code in the form of a function, just make sure your variable names match what I've described above so the tests work.

In [2]:
import pandas as pd
# This is just to show you the name to use for the variable you need to create for this step to pass.
all_hospitals = pd.read_csv('complications_all.csv')


In [3]:
print(all_hospitals.columns)       # confirm column names
all_hospitals.head()


Index(['Facility ID', 'Facility Name', 'Address', 'City', 'State', 'ZIP Code',
       'County Name', 'Phone Number', 'Measure ID', 'Measure Name',
       'Compared to National', 'Denominator', 'Score', 'Lower Estimate',
       'Higher Estimate', 'Footnote', 'Start Date', 'End Date'],
      dtype='object')


,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Measure ID,Measure Name,Compared to National,Denominator,Score,Lower Estimate,Higher Estimate,Footnote,Start Date,End Date
0,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,COMP_HIP_KNEE,Rate of complications for hip/knee replacement...,No Different Than the National Rate,292,3.2,2.1,4.8,NaN,04/01/2015,03/31/2018
1,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,MORT_30_AMI,Death rate for heart attack patients,No Different Than the National Rate,688,13,11.0,15.5,NaN,07/01/2015,06/30/2018
2,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,MORT_30_CABG,Death rate for CABG surgery patients,No Different Than the National Rate,291,4.3,2.6,6.8,NaN,07/01/2015,06/30/2018
3,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,MORT_30_COPD,Death rate for COPD patients,No Different Than the National Rate,411,8.8,6.7,11.4,NaN,07/01/2015,06/30/2018
4,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,MORT_30_HF,Death rate for heart failure patients,No Different Than the National Rate,869,12.7,10.7,15.0,NaN,07/01/2015,06/30/2018


In [4]:
# 3) Filter to Missouri hospitals
mo_hospitals = all_hospitals[all_hospitals['State'] == 'MO'].copy()
# quick sanity check
print("MO rows:", len(mo_hospitals))

MO rows: 2133


In [6]:
# 4) Clean the denominator column:
# - Remove rows where denominator is 'Not Available' (case-sensitive)
# - Remove commas or other formatting, then convert to numeric
mask_available = mo_hospitals['Denominator'].astype(str).str.strip().str.upper() != 'NOT AVAILABLE'
mo_hospitals = mo_hospitals[mask_available].copy()

# Remove commas / spaces and convert to numeric (coerce errors -> NaN)
mo_hospitals['Denominator'] = (
    mo_hospitals['Denominator']
    .astype(str)
    .str.replace(',', '')   # remove thousands separators if any
    .str.strip()
)
mo_hospitals['Denominator'] = pd.to_numeric(mo_hospitals['Denominator'], errors='coerce')

# Drop any rows where conversion failed (NaN)
mo_hospitals = mo_hospitals[mo_hospitals['Denominator'].notna()].copy()

In [7]:
# 5) Convert date columns to pandas datetime
# Use the actual column names in your file. Common names: measure_start_date, measure_end_date
mo_hospitals['Start Date'] = pd.to_datetime(mo_hospitals['Start Date'], errors='coerce')
mo_hospitals['End Date']   = pd.to_datetime(mo_hospitals['End Date'], errors='coerce')

# If any dates failed to parse, you might want to inspect:
bad_dates = mo_hospitals[mo_hospitals['Start Date'].isna() | mo_hospitals['End Date'].isna()]
print("Rows with unparsable dates:", len(bad_dates))

Rows with unparsable dates: 0


In [8]:
# 6) Aggregate by hospital to create mo_summary
mo_summary = (
    mo_hospitals
    .groupby('Facility Name')
    .agg(
        start_date=('Start Date', 'min'),
        end_date  =('End Date', 'max'),
        number    =('Denominator', 'sum')
    )
)

# Keep index as hospital_name (which is what the tests expect)
# Optionally sort index alphabetically for readability
mo_summary = mo_summary.sort_index()

In [ ]:
all_hospitals = pd.read_csv('complications_all.csv')

In [ ]:
all_hospitals['State'] = all_hospitals['State'].astype(str).str.strip().str.upper()

In [9]:
mo_hospitals['Facility Name'] = mo_hospitals['Facility Name'].str.strip().str.upper()

In [10]:
print(mo_summary.columns.tolist())

['start_date', 'end_date', 'number']


In [11]:
mo_summary.columns = mo_summary.columns.str.strip().str.title()
print(mo_summary.columns)

Index(['Start_Date', 'End_Date', 'Number'], dtype='object')


In [ ]:
assert(mo_summary['Number'].sum() == 1766908)


In [12]:
print(mo_summary.head())

                                   Start_Date   End_Date  Number
Facility Name                                                   
BARNES JEWISH HOSPITAL             2015-04-01 2018-06-30  131313
BARNES-JEWISH ST PETERS HOSPITAL   2015-04-01 2018-06-30   15668
BARNES-JEWISH WEST COUNTY HOSPITAL 2015-04-01 2018-06-30    9622
BATES COUNTY MEMORIAL HOSPITAL     2015-07-01 2018-06-30    3117
BELTON REGIONAL MEDICAL CENTER     2015-04-01 2018-06-30    9270


In [13]:
import pandas as pd

assert(mo_summary['Number'].sum() == 1766908)
assert(mo_summary['Start_Date'].min() == pd.Timestamp(2015,4,1))
assert(mo_summary['End_Date'].max() == pd.Timestamp(2018,6,30))
assert(mo_summary.shape == (108,3))
assert(mo_summary.loc['BARNES JEWISH HOSPITAL']['Number'] == 131313)
assert(mo_summary.loc['BOONE HOSPITAL CENTER']['Number'] == 63099)

print("All asserts passed — mo_summary looks correct.")

All asserts passed — mo_summary looks correct.


In [15]:
import pandas as pd

all_hospitals = pd.read_csv('complications_all.csv')

# normalize state & hospital name
all_hospitals['State'] = all_hospitals['State'].astype(str).str.strip().str.upper()
all_hospitals['Facility Name'] = all_hospitals['Facility Name'].astype(str).str.strip().str.upper()

mo_hospitals = all_hospitals[all_hospitals['State'] == 'MO'].copy()

# denominator clean
mo_hospitals = mo_hospitals[mo_hospitals['Denominator'].astype(str).str.strip().str.upper() != 'NOT AVAILABLE'].copy()
mo_hospitals['Denominator'] = mo_hospitals['Denominator'].astype(str).str.replace(',', '').str.strip()
mo_hospitals['Denominator'] = pd.to_numeric(mo_hospitals['Denominator'], errors='coerce')
mo_hospitals = mo_hospitals[mo_hospitals['Denominator'].notna()].copy()

# dates
mo_hospitals['Start Date'] = pd.to_datetime(mo_hospitals['Start Date'], errors='coerce')
mo_hospitals['End Date']   = pd.to_datetime(mo_hospitals['End Date'], errors='coerce')

# aggregate
mo_summary = mo_hospitals.groupby('Facility Name').agg(
    start_date=('Start Date', 'min'),
    end_date  =('End Date', 'max'),
    number    =('Denominator', 'sum')
).sort_index()

In [16]:
print(mo_summary)

                                    start_date   end_date  number
Facility Name                                                    
BARNES JEWISH HOSPITAL              2015-04-01 2018-06-30  131313
BARNES-JEWISH ST PETERS HOSPITAL    2015-04-01 2018-06-30   15668
BARNES-JEWISH WEST COUNTY HOSPITAL  2015-04-01 2018-06-30    9622
BATES COUNTY MEMORIAL HOSPITAL      2015-07-01 2018-06-30    3117
BELTON REGIONAL MEDICAL CENTER      2015-04-01 2018-06-30    9270
...                                        ...        ...     ...
TRUMAN MEDICAL CENTER LAKEWOOD      2015-04-01 2018-06-30    4297
UNIVERSITY OF MISSOURI HEALTH CARE  2015-04-01 2018-06-30   56493
WASHINGTON COUNTY MEMORIAL HOSPITAL 2015-07-01 2018-06-30     220
WESTERN MISSOURI MEDICAL CENTER     2015-04-01 2018-06-30    7254
WRIGHT MEMORIAL HOSPITAL            2015-07-01 2018-06-30     198

[108 rows x 3 columns]


In [ ]:
assert(mo_summary['number'].sum() == 1766908)
assert(mo_summary['start_date'].min() == pd.Timestamp(2015,4,1))
assert(mo_summary['end_date'].max() == pd.Timestamp(2018,6,30))
assert(mo_summary.shape == (108,3))
assert(mo_summary.loc['BARNES JEWISH HOSPITAL'].number == 131313)
assert(mo_summary.loc['BOONE HOSPITAL CENTER'].number == 63099)

---

### 47.2 Planning your final project

You should be thinking about the things we've been learning and how you can apply them to your final project.  Use the rubric to help guid your thinking and then answer the questions below.  This is meant as a guide to help you think through what you will do.

#### A) Data Access

Your project should include data from at least three distinct types of sources.  For example: AWS S3, Relational Databases, Internet, Web Services, local files.  List what data sources you're planning to use.

**Double-click to enter your answer**


I am planning to use AWS S3, Relational Databases and Local Files in my final project.

AWS S3: S3 bucket in AWS is used to store and retrieve the large amount of data sets it can be of any kind like images, documents or log files.

Relational Databases: In relational databases we have like MySQL, SQLite and PostgreSQL these are used to store the data in structured format. Mainly the relational data can be stored using this.

Local Files: When we download the dataset from an external source the file will be saved in the .CSV format in our local system or when we create a dataset using the real time data that we collected that also we are saving with .CSV extinction in our system.

These are the main three types of data that i am going to use in my final project.

#### B. Data Formats

Your project should include data that comes in different file formats.  For example: HL7, EDI, HTML, CSV, Excel, JSON, XML.  List what data formats you're planning to use.

**Double-click to enter your answer**

I am planning to use CSV, XML and JSON files for my final project.

CSV (Comma-Separated Values): CSV files are widely used and simple to create and perform the tabular operations on the data.

XML (extensible Mark-up Language): It is a mark-up language used to store and transfer the data easily.

JSON (JavaScript Object Nation): Mainly JSON files are used for representing the structured data, we will be working with the structured data so it is useful.

These are the three data formats I am going to use in my final project.


#### C. Objective

What purpose would your project serve in a real work setting?  Take a couple of paragraphs to write down why this is an interesting product.

**Double-click to enter your answer**
This project providеs a flеxiblе solution for businеssеs facing data intеgration challеngеs by intеgrating data from AWS S3, Intеrnеt sourcеs, and local filеs in a variеty of formats such as HTML, CSV, and Excеl. Many organizations work with dispеrsеd data on multiplе platforms еncodеd in multiplе formats, making it difficult to intеgratе and analyzе data еasily Manufacturing strеamlinеs data prеprocеssing phasеs and improvеs thе accuracy and prеcision of analytics work morе еfficiеntly using a platform that can еasily managе thеsе typеs of data sourcеs and formats.

This projеct's practical valuе is dеmonstratеd by its ability to еnhancе organizational dеcision-making. Businеssеs can gain insights from a largеr rangе of data with thе hеlp of a flеxiblе and unifiеd data intеgration tool, which makеs wеll-informеd dеcision-making еasiеr. This in turn fostеrs rеsponsivеnеss and agility, which arе еssеntial traits for businеssеs navigating fast-pacеd, cutthroat markеts. Thе projеct makеs a significant contribution to a morе еfficiеnt and succеssful businеss intеlligеncе stratеgy by offеring an еasy-to-usе intеrfacе for data aggrеgation and analysis. This makеs it possiblе for businеssеs to fully utilizе thе data thеy havе, which hеlps with opеrational and stratеgic planning.




---



## Submit your work via GitHub as normal
